<img align="right" style="padding-left:10px; height: 60%; width: 60%" src="figures/use_cases.jpg" >

# Recommendations

Recommendations systems have taken on an important role in Data Science 

### Movie Suggestion:

We will start this example off by comparing different movies, then recommending similar ones. First, import a sample dataset.

In [ ]:
import pandas as pd
from pandas import DataFrame

movies = pd.read_csv('MoviesList.csv',header = 0) 
movies

Take a look at what the columns are. Since we don't need all of them, we can make a new dataframe with just the ones that we want. 

In [ ]:
movies.columns

We will be using _Rating_ and the two _Genre_ columns to come up with recommendations.

In [ ]:
movie_attrs = DataFrame(movies, columns = ['Movie ID:', 'Name:','Genre:','Rating:','Additional Genre:'])

In [ ]:
# when "Additional Genre:" is blank...
movie_attrs.loc[(movie_attrs['Additional Genre:'].isna()), 'Additional Genre:'] = ''
# movie_attrs[40:44]

In [ ]:
fave_movie = 5 # The index of the user's favorite movie (Titanic)

In [ ]:
def combos_no_dups(left, right):
    combos = left.assign(key=1).merge(right.assign(key=1), on='key').drop('key', 1)
    combos = combos[ (combos['Movie ID:_x'] != combos["Movie ID:_y"]) ]
    return combos

movie_attrs.apply((lambda x: (x['Name:'], x['Genre:'], x['Additional Genre:'], x['Rating:'])), axis=1)

combos = combos_no_dups(movie_attrs[fave_movie:fave_movie+1], movie_attrs[:])
#combos

### Jaccard Similarity

<div>
    <img align="right" style="padding-left:10px; height: 20%; width: 20%" src="figures/jaccard_union.png" >
</div><div>
    <img align="right" style="padding-left:10px; height: 20%; width: 20%" src="figures/jaccard_intersection.png" >
</div><div>
    <br /><img align="left" style="padding-right:10px; height: 40%; width: 40%" src="figures/jaccard_equation.png" >
</div>

### Jaccard Similarity Implementation:

The implementation of the similarity function is based on Python `sets`, the only catch being that `Additional Genre` can sometimes be blank, throwing off the calculation for those situations.

In [ ]:
def jaccard_similarity(list1, list2):
    set1_ = set(filter(None, list1)) # remove any blanks
    set2_ = set(filter(None, list2))
    intersection = set1_ & set2_
    union = set1_ | set2_
    similarity = float(len(intersection) / len(union))
    return float(similarity)

In [ ]:
def similarity(row):
    list1 = [row['Genre:_x'], row['Rating:_x'], row['Additional Genre:_x']]
    list2 = [row['Genre:_y'], row['Rating:_y'], row['Additional Genre:_y']]
    return jaccard_similarity(list1, list2)

combos['similarity'] = combos.apply(similarity, axis=1)

drop_columns = ['Movie ID:_x', 'Name:_x', 'Genre:_x', 'Rating:_x',
                'Additional Genre:_x', 'Movie ID:_y', 'Genre:_y', 'Rating:_y', 'Additional Genre:_y']

In [ ]:
print ('If you liked {}, you might also enjoy'.format(list(movie_attrs[fave_movie:fave_movie+1]['Name:'])[0]))

# Set the threshold to 0.22 to verify that we caught cases where Additional Genre was blank.

combos[combos['similarity'] > 0.22].drop(columns=drop_columns)

## The wrong approach 

If you are coming from a traditional programming language background, you may be tempted to write code with loops in it, as shown below. _Do not yield to this temptation!_ 
The point of libraries like numpy and pandas is to exploit the parallelism inherent in `ndarray`s and `dataframe`s. Both libraries take advantage of optimizations that your "loopy" code can not.

In [ ]:
# Avoid this type of coding:

# def movie_suggestion(movie_id):
#     num_movies = len(movie_attrs)
#     print('If you liked: ' + movie_attrs.iloc[movie_id-1]['Name:'] + '\n')
#     print('Similar movies that you would like are:\n')
#     l1 = [movie_attrs.iloc[movie_id-1]['Genre:'],movie_attrs.iloc[movie_id - 1]['Additional Genre:'],movie_attrs.iloc[movie_id - 1]['Rating:']]
#     for counter in range(0,num_movies):
#         l2 = [movie_attrs.iloc[counter]['Genre:'],movie_attrs.iloc[counter]['Additional Genre:'],movie_attrs.iloc[counter]['Rating:']]
#         x = jaccard_similarity(l1,l2) 
#         if (x > .22):
#             if (counter != movie_id - 1):
#                 print(movie_attrs.iloc[counter]['Name:'], x)

<a href="https://en.wikipedia.org/wiki/Collaborative_filtering"><img align="right" style="padding-left:10px; height: 40%; width: 40%" src="https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif" ></a>

The above example is an example of **Collaborative filtering**, a kind of recommendation system. This type of recommendation system makes predictions of what might interest a person based on the taste of many other users. It assumes that if person X likes Snickers, and person Y likes Snickers and Milky Way, then person X might like Milky Way as well. This version of collaborative filtering is close to clustering &mdash; the recommendations are points in hyperspace that are close to the initial points.

A generalized version of Collaborative filtering, implied by the image above, is a three-step process:

1. A user expresses their preferences by rating items (e.g. books, movies or CDs) of the system. These ratings can be viewed as an approximate representation of the user's interest in the corresponding domain.
2. The system matches this user's ratings against other users' and finds the people with most "similar" tastes.
3. With similar users, the system recommends items that the similar users have rated highly but not yet being rated by this user (presumably the absence of rating is often considered as the unfamiliarity of an item)

A key problem of collaborative filtering is how to combine and weight the preferences of user neighbors. Sometimes, users can immediately rate the recommended items. As a result, the system gains an increasingly accurate representation of user preferences over time.

Recommendation systems based on **Content-based filtering** focus on the products themselves and recommends other products that have similar attributes. Content-based filtering relies on the characteristics of the products themselves, not on _other_ users to interact with the products before making a recommendation.

Experience has shown <a href="https://en.wikipedia.org/wiki/Recommender_system#Hybrid_recommender_systems"><strong>Hybrid Recommendation Systems</strong></a> to be the most effective but they are the most expensive to build and/or deploy.

# A more extensive dataset

Collaborative datasets for movies (and other products) can be large! Here is a [small (1 MB) subset of the IMDB database](https://grouplens.org/datasets/movielens/latest/), downloaded and unzipped for your convenience.

I'm not sure if our servers can accommodate everyone working with the dataset simultaneously but it's worth trying. 

The above-referenced page also links to the _full_ version of the dataset (265MB, zipped). We will **not** be using that dataset in this class.

In [ ]:
movies = pd.read_csv('ml-latest-small/movies.csv',header = 0) 

movies

In [ ]:
ratings = pd.read_csv('ml-latest-small/ratings.csv',header = 0) 
ratings

You will need to interpret the [timestamp](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html) thus:
```
    >>> pd.Timestamp(1493846415, unit='s')
    Timestamp('2017-05-03 21:20:15')
```

We will be covering the topic of movie selection further in **05-02-movie-suggestion**, time permitting.